In [1]:
import pandas as pd

train_df = pd.read_csv('../data/train_fp.csv', index_col='Unnamed: 0')
test_df = pd.read_csv('../data/test_fp.csv', index_col='Unnamed: 0')
print("Train DF shape: {}".format(train_df.shape),
      "Test DF shape: {}".format(test_df.shape))


Train DF shape: (514, 4097) Test DF shape: (128, 4097)


In [2]:
X_train = train_df.iloc[:,:-1]
y_train = train_df.label.values

X_test = test_df.iloc[:,:-1]
y_test = test_df.label.values

### Test FFNN

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
import random


In [4]:
# Create dataset class
class BinaryMoleculeDataset(Dataset):
    
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X.astype(bool).astype(float))
        self.y = torch.FloatTensor(y).reshape(-1, 1)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]


In [5]:

# Create FFNN for binary input
class BinaryFFNN(nn.Module):

    def __init__(self,
                 input_size,
                 hidden_sizes=[2048, 1024, 512],
                 dropout_rate=0.2):
        super(BinaryFFNN, self).__init__()

        layers = []

        # Input layer
        layers.append(nn.Linear(input_size, hidden_sizes[0]))
        layers.append(nn.ReLU())
        layers.append(nn.BatchNorm1d(hidden_sizes[0]))
        layers.append(nn.Dropout(dropout_rate))

        # Hidden layers
        for i in range(len(hidden_sizes)-1):
            layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
            layers.append(nn.ReLU())
            layers.append(nn.BatchNorm1d(hidden_sizes[i+1]))
            layers.append(nn.Dropout(dropout_rate))
        
        # Output layer
        layers.append(nn.Linear(hidden_sizes[-1], 1))

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


In [6]:
# Create training function with early stopping and learning rate

def train_binary_model(model,
                       train_loader,
                       val_loader,
                       criterion,
                       optimizer,
                       num_epochs=100,
                       patience=20,
                       device='cuda'):
    
    model = model.to(device)
    best_val_loss = float('inf')
    patience_counter = 0
    train_losses = []
    val_losses = []

    # Add learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5
    )

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        
        for X_batch, y_batch in train_loader:
            
            X_batch, y_batch= X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            ouputs = model(X_batch)
            loss = criterion(ouputs, y_batch)
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(),
                                          max_norm=1)
            
            optimizer.step()
            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)
            
        # Validation phase
        model.eval()
        val_loss= 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        # Learning rate scheduling
        scheduler.step(val_loss)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            # torch.save(model.state_dict(),
            #            'best_binary_model.pt')
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
        #     print(f'Early stopping at epoch {epoch}')
            break

        # if epoch % 10 == 0:
        #     print(f'Epoch {epoch}: Train Loss = {train_loss:.4f}, Val loss = {val_loss:.4f}')

    return train_losses, val_losses

In [7]:
def main(X,
         y,
         hidden_sizes=[2048, 1024, 512],
         dropout_rate=0.2,
         patience=20,
         lr=0.001,
         test_size=0.2,
         batch_size=32):
    
    X = X.astype(bool).astype(float)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # Create dataloaders
    train_dataset = BinaryMoleculeDataset(X_train, y_train)
    test_dataset = BinaryMoleculeDataset(X_test, y_test)

    train_loader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=True)
    test_loader = DataLoader(test_dataset,
                             batch_size=batch_size)
    
    # Initialize model
    input_size = X_train.shape[1]
    model = BinaryFFNN(input_size,
                       hidden_sizes,
                       dropout_rate)

    # Initialize loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=lr,
                                 weight_decay=1e-5)
    
    # Train model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_losses, val_losses = train_binary_model(model,
                                                  train_loader,
                                                  test_loader,
                                                  criterion,
                                                  optimizer,
                                                  patience=patience,
                                                  device=device)
    return model, train_losses, val_losses

In [8]:
def evaluate_model(model,
                   X_test,
                   y_test,
                   batch_size=32,
                   device='cuda',
                   print=False):
    # Evaluate model and print metrics
    model.eval()
    test_dataset = BinaryMoleculeDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    predictions = []
    actuals = []

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            outputs = model(X_batch)
            predictions.extend(outputs.cpu().numpy())
            actuals.extend(y_batch.numpy())

    predictions = np.array(predictions)
    actuals = np.array(actuals)

    mse = np.mean((predictions - actuals) ** 2)
    rmse = np.sqrt(mse)
    mae = np.mean(np.abs(predictions - actuals))
    r2 = 1 - np.sum((actuals - predictions) ** 2) / np.sum((actuals- np.mean(actuals)) ** 2)

    if print:
        print("Test results:")
        print(f"MSE: {mse:.3f}")
        print(f"R²: {r2:.3f}")

    return r2

In [47]:
#Set seeds
def set_seeds(seed=42):
    """Set all random seeds for reproducibility"""
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # for multi-GPU
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Create train_eval function for hyperparameter optimization
def train_evaluate(parametrization):
    set_seeds()
    hidden_sizes = [int(x) for x in parametrization['hidden_sizes'].strip('[]').split(',')]
    parametrization.pop('hidden_sizes')
    fnn_model, _, _ = main(X_train.to_numpy(),
                           y_train,
                           hidden_sizes,
                           **parametrization)
    return evaluate_model(fnn_model, X_test.to_numpy(), y_test)

In [41]:
from ax.service.ax_client import AxClient, ObjectiveProperties
from ax.service.utils.report_utils import exp_to_df
from ax.utils.notebook.plotting import init_notebook_plotting, render

In [42]:
ax_client = AxClient()

[INFO 10-28 14:41:31] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.


In [43]:
# hidden_sizes=[2048, 1024, 512],
# dropout_rate=0.2,
# patience = 20
# lr = 0.001

# Create an experiment with required arguments: name, parameters, and objective_name.
ax_client.create_experiment(
    name="ffnn_hyperparameter_search",  # The name of the experiment.
    parameters=[
        {
            "name": "lr",  # The name of the parameter.
            "type": "range",  # The type of the parameter ("range", "choice" or "fixed").
            "bounds": [1e-5, 1],  # The bounds for range parameters. 
            # "values" The possible values for choice parameters .
            # "value" The fixed value for fixed parameters.
            "value_type": "float",  # Optional, the value type ("int", "float", "bool" or "str"). Defaults to inference from type of "bounds".
            "log_scale": True,  # Optional, whether to use a log scale for range parameters. Defaults to False.
            # "is_ordered" Optional, a flag for choice parameters.
        },
        {
            "name": "patience",  
            "type": "range",  
            "bounds": [5, 20],
            "value_type": "int" 
        },
        {
            "name": "dropout_rate",
            "type": "range",
            "bounds": [1e-2, 0.5],
            "value_type": "float",
        },
        {
            "name": "hidden_sizes",
            "type": "choice",
            "values": [
                "[2048, 1024, 512]",
                "[2048, 1024, 512, 256]",
                "[1024, 512, 256]",
                "[4096, 2048, 1024, 512]",
                "[2048, 2048, 1024]",
            ],
            "value_type": "str",
        },
    ],
    objectives={"r2": ObjectiveProperties(minimize=False)},  # The objective name and minimization setting.
    # parameter_constraints: Optional, a list of strings of form "p1 >= p2" or "p1 + p2 <= some_bound".
    # outcome_constraints: Optional, a list of strings of form "constrained_metric <= some_bound".
    overwrite_existing_experiment=True,
)

/home/papafrita/projects/solvation_free_energy/.venv/lib/python3.11/site-packages/ax/service/utils/instantiation.py:248: AxParameterWarning:

`is_ordered` is not specified for `ChoiceParameter` "hidden_sizes". Defaulting to `False`  since the parameter is a string with more than 2 choices.. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction. Note that choice parameters with exactly 2 choices are always considered ordered and that the user-supplied `is_ordered` has no effect in this particular case.

/home/papafrita/projects/solvation_free_energy/.venv/lib/python3.11/site-packages/ax/service/utils/instantiation.py:248: AxParameterWarning:

`sort_values` is not specified for `ChoiceParameter` "hidden_sizes". Defaulting to `False` for parameters of `ParameterType` STRING. To override this behavior (or avoid this warning), specify `sort_values` during `ChoiceParameter` construction.

[INFO 10-28 14:41:33] ax.service.utils.instantia

In [44]:
ax_client.attach_trial(
    parameters={"lr":0.01,
                "patience": 10,
                "dropout_rate": 0.2,
                "hidden_sizes":"[2048, 1024, 512]"}
)

[INFO 10-28 14:41:35] ax.core.experiment: Attached custom parameterizations [{'lr': 0.01, 'patience': 10, 'dropout_rate': 0.2, 'hidden_sizes': '[2048, 1024, 512]'}] as trial 0.


({'lr': 0.01,
  'patience': 10,
  'dropout_rate': 0.2,
  'hidden_sizes': '[2048, 1024, 512]'},
 0)

In [48]:
baseline_parameters = ax_client.get_trial_parameters(trial_index=0)
ax_client.complete_trial(trial_index=0, raw_data=train_evaluate(baseline_parameters))

[INFO 10-28 14:42:20] ax.service.ax_client: Completed trial 0 with data: {'r2': (0.863856, None)}.


In [49]:
for i in range(25):
    parameters, trial_index = ax_client.get_next_trial()
    # Local evaluation here can be replaced with deployment to external system.
    ax_client.complete_trial(trial_index=trial_index, raw_data=train_evaluate(parameters))

/home/papafrita/projects/solvation_free_energy/.venv/lib/python3.11/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning:

Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.

[INFO 10-28 14:42:26] ax.service.ax_client: Generated new trial 1 with parameters {'lr': 0.012951, 'patience': 12, 'dropout_rate': 0.268742, 'hidden_sizes': '[2048, 1024, 512, 256]'} using model Sobol.
[INFO 10-28 14:42:29] ax.service.ax_client: Completed trial 1 with data: {'r2': (0.831964, None)}.
/home/papafrita/projects/solvation_free_energy/.venv/lib/python3.11/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning:

Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.

[INFO 10-28 14:42:29] ax.service.ax_client: Generated new trial 2 with parameters {'lr': 2.6e-05, 'patience': 17, 'dropout_rate': 0.206178, 'hidden_sizes': '[1024, 512, 256]'} using model Sobol.
[INFO 10-28 14:42:36] ax.ser

In [50]:
ax_client.get_trials_data_frame()

[WARNING 10-28 14:50:41] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


,trial_index,arm_name,trial_status,generation_method,r2,lr,patience,dropout_rate,hidden_sizes
0,0,0_0,COMPLETED,Manual,0.863856,0.010000,10,0.200000,"[2048, 1024, 512]"
1,1,1_0,COMPLETED,Sobol,0.831964,0.012951,12,0.268742,"[2048, 1024, 512, 256]"
2,2,2_0,COMPLETED,Sobol,-0.075728,0.000026,17,0.206178,"[1024, 512, 256]"
3,3,3_0,COMPLETED,Sobol,0.788323,0.000630,6,0.438754,"[1024, 512, 256]"
4,4,4_0,COMPLETED,Sobol,0.863911,0.156510,15,0.012537,"[2048, 1024, 512]"
5,5,5_0,COMPLETED,Sobol,-5.819693,0.865897,7,0.186945,"[2048, 1024, 512]"
6,6,6_0,COMPLETED,Sobol,0.833756,0.002023,14,0.372014,"[4096, 2048, 1024, 512]"
7,7,7_0,COMPLETED,Sobol,0.801181,0.000174,9,0.123622,"[2048, 1024, 512]"
8,8,8_0,COMPLETED,Sobol,0.854067,0.034821,20,0.427380,"[2048, 2048, 1024]"
9,9,9_0,COMPLETED,BO_MIXED,0.881123,0.019200,16,0.040558,"[2048, 1024, 512]"


In [61]:
best_parameters, values = ax_client.get_best_parameters()
best_parameters

{'lr': 0.019199806049574764,
 'patience': 16,
 'dropout_rate': 0.04055798755135649,
 'hidden_sizes': '[2048, 1024, 512]'}

In [52]:
mean, covariance = values
mean

{'r2': 0.8800653924964192}

In [58]:
render(ax_client.get_contour_plot(param_x="dropout_rate", param_y="lr", metric_name="r2"))

[INFO 10-28 14:52:22] ax.service.ax_client: Retrieving contour plot with parameter 'dropout_rate' on X-axis and 'lr' on Y-axis, for metric 'r2'. Remaining parameters are affixed to the middle of their range.


In [54]:
render(
    ax_client.get_optimization_trace()
)  

In [55]:
ax_client.get_trials_data_frame().to_csv("../data/optimization_results/FFNN_optimization.csv")

[WARNING 10-28 14:50:49] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


In [68]:
set_seeds()

optimal_parameters = best_parameters.copy()
opt_hidden_sizes = [int(x) for x in optimal_parameters['hidden_sizes'].strip('[]').split(',')]
optimal_parameters.pop('hidden_sizes')


'[2048, 1024, 512]'

In [70]:
optimal_parameters

{'lr': 0.019199806049574764,
 'patience': 16,
 'dropout_rate': 0.04055798755135649}

In [73]:
best_fnn_model, train_losses, val_losses = main(X_train.to_numpy(),
                                                y_train,
                                                hidden_sizes=opt_hidden_sizes,
                                                **optimal_parameters)

evaluate_model(best_fnn_model,
               X_test.to_numpy(),
               y_test)

0.8811232000589371